网页浏览行为关联规则挖掘

任务: 分析用户在网站上的浏览行为数据，挖掘潜在的跳转规律，为网站提供优化导航结构的建议。

数据集: UCI ML Repository - Anonymous Microsoft Web Data，该数据为“来自 www.microsoft.com 的匿名网络数据”，通过对 www.microsoft.com 日志进行采样和处理来创建数据。该数据记录了38000名匿名、随机选择的用户使用 www.microsoft.com。对于每个用户，数据列出了用户在一周内访问的网站（Vroots）的所有区域。


数据集格式说明：
数据采用基于 ASCII 的稀疏数据格式，数据文件的每一行都以一个字母开头，其中开头字母‘A’表示网站属性，数据包含5个字段，分别为A属性、Vroot属性ID、可忽略的数字、Vroot标题、Vroot网站网址。
对于每个用户，都有一个case行，后跟零个或多个vote行，
C,"10164",10164           表示ID为10164的用户网站访问情况
V,1123,1
V,1009,1
V,1052,1                表示ID为10164的用户访问网站的属性ID。

由此可见，数据集分为网站数据、用户数据、用户访问网站数据。

1.数据预处理: 清洗数据，处理缺失值，提取用户浏览记录。
1.1 数据预处理
 
 对数据集进行处理，分别获取网站数据、用户数据、用户访问网站数据

In [2]:
import pandas as pd
import os.path as osp
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
root_path = "./datasets"

#读取数据
with open('./datasets/anonymous-msweb.data', 'r') as f:
   data = f.readlines()
##选择A开头的网站数据，存入Web_data中
##选择C开头的用户数据，存入user_data中
##在V开头的数据后，添加V行的用户ID信息，存入visit_data，表示用户访问网站情况
a_lines = [line for line in data if line.startswith('A')]
c_lines = [line for line in data if line.startswith('C')]
a_data = pd.DataFrame([x.split(',') for x in a_lines], columns=['Type', 'ID', 'ignore', 'Title', 'Stream'])
c_data = pd.DataFrame([x.split(',') for x in c_lines], columns=['Type', 'UserID', 'num'])
ac_lines = []
for line in data:
    if line.startswith('C')|line.startswith('V'):
        ac_lines.append(line)
##将C、V开头的数据整合为以V开头的数据，将用户信息添加到V开头的数据后
result = []
temp = []
for line in ac_lines:
    if line.startswith('C'):
        if temp:
            result.append(temp)
        temp = [line]
    else:
        temp.append(line)
if temp:
    result.append(temp)

fw_data = []
for group in result:
    c_line1 = group[0].split(',')
    for v_line1 in group[1:]:
        fw_data.append(v_line1 + ',' + c_line1[1])
fw_dt = pd.DataFrame([x.split(',') for x in fw_data], columns=['Type', 'UserID', 'ign', 'ID'])

a_attri = ['ID', 'Title', 'Stream']
c_attri = ['UserID']
fw_attri = ['UserID', 'ID']

web_data = a_data[a_attri]
print("网站数据如下：\n",web_data)

网站数据如下：
        ID                              Title           Stream
0    1287          "International AutoRoute"   "/autoroute"\n
1    1288                          "library"     "/library"\n
2    1289  "Master Chef Product Information"  "/masterchef"\n
3    1297                  "Central America"    "/centroam"\n
4    1215         "For Developers Only Info"   "/developer"\n
..    ...                                ...              ...
289  1219    "Corporate Advertising Content"         "/ads"\n
290  1030                "Windows NT Server"    "/ntserver"\n
291  1182                          "Fortran"     "/fortran"\n
292  1100                  "MS in Education"   "/education"\n
293  1210                      "SNA Support"  "/snasupport"\n

[294 rows x 3 columns]


In [3]:
user_data = c_data[c_attri]
print("用户数据如下：\n",user_data)

用户数据如下：
         UserID
0      "10001"
1      "10002"
2      "10003"
3      "10004"
4      "10005"
...        ...
32706  "42707"
32707  "42708"
32708  "42709"
32709  "42710"
32710  "42711"

[32711 rows x 1 columns]


In [5]:
visit_data =fw_dt[fw_attri]
print("用户访问网站数据如下：\n",visit_data)

用户访问网站数据如下：
       UserID       ID
0       1000  "10001"
1       1001  "10001"
2       1002  "10001"
3       1001  "10002"
4       1003  "10002"
...      ...      ...
98649   1003  "42709"
98650   1035  "42710"
98651   1001  "42710"
98652   1018  "42710"
98653   1008  "42711"

[98654 rows x 2 columns]


In [ ]:
根据数据处理结果可以看出，该训练数据集中网站共有294个，以ID、网站Title、网站Stream标识；用户共有32711个，以UserID标识；用户访问网站记录共有98654条，以UserID、网站ID标识。

1.2 缺失值处理

In [11]:
print('网站数据缺失值情况： \n',web_data.isna().sum())
print('用户数据缺失值情况： \n',user_data.isna().sum())
print('用户访问网站数据缺失值情况： \n',visit_data.isna().sum())

网站数据缺失值情况： 
 ID        0
Title     0
Stream    0
dtype: int64
用户数据缺失值情况： 
 UserID    0
dtype: int64
用户访问网站数据缺失值情况： 
 UserID    0
ID        0
dtype: int64


统计发现，数据中不存在缺失值。

1.3 提取用户浏览记录

In [12]:
visit_data =fw_dt[fw_attri]
print("用户访问网站数据如下：\n",visit_data)

用户访问网站数据如下：
       UserID       ID
0       1000  "10001"
1       1001  "10001"
2       1002  "10001"
3       1001  "10002"
4       1003  "10002"
...      ...      ...
98649   1003  "42709"
98650   1035  "42710"
98651   1001  "42710"
98652   1018  "42710"
98653   1008  "42711"

[98654 rows x 2 columns]


经数据预处理得出的visit_data即为用户浏览记录，上述输出结果中UserId为用户的ID信息，ID为当前用户访问的网站ID信息。

2.  数据探索性分析: 分析最常被访问的页面、页面访问量分布等。

In [22]:
ID_counts = visit_data["ID"].value_counts()
print('页面访问量分布：\n',ID_counts)

页面访问量分布：
 "40310"    35
"25185"    31
"12147"    30
"10348"    28
"25922"    28
           ..
"31467"     1
"31471"     1
"31472"     1
"31473"     1
"42711"     1
Name: ID, Length: 32711, dtype: int64


上述统计结果表示，左侧为网站的ID，右侧为用户访问次数。

In [21]:
print('最常被访问的页面：\n',ID_counts.head(10))

最常被访问的页面：
 "40310"    35
"25185"    31
"12147"    30
"10348"    28
"25922"    28
"12815"    28
"41066"    28
"19860"    26
"31809"    26
"40122"    24
Name: ID, dtype: int64


上述统计结果表示，最常被访问的10个网站页面的网站ID值为40310、25185、12147、10348、25922、12815、41066、19860、31809、40122。

3.  关联规则挖掘: 使用Apriori算法根据用户浏览记录计算频繁项集和关联规则。
频繁项集和关联规则如下

In [25]:
visit_data["ID"] = visit_data["ID"].str.strip('"')
basket = (visit_data.groupby(["UserID", "ID"])["ID"]
          .count().unstack().reset_index().fillna(0)
          .set_index("UserID"))

frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

print("频繁项集：")
print(frequent_itemsets)

print("关联规则：")
print(rules)

C:\Users\59554\AppData\Local\Temp\ipykernel_15916\1585013422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visit_data["ID"] = visit_data["ID"].str.strip('"')


频繁项集：
      support        itemsets
0    0.059649         (10021)
1    0.080702         (10068)
2    0.056140         (10132)
3    0.059649         (10156)
4    0.063158         (10208)
..        ...             ...
161  0.052632  (33341, 25922)
162  0.056140  (40310, 25922)
163  0.052632  (41066, 25922)
164  0.056140  (36955, 40310)
165  0.052632  (41066, 40310)

[166 rows x 2 columns]
关联规则：
   antecedents consequents  antecedent support  consequent support   support  \
0      (10348)     (12147)            0.098246            0.105263  0.056140   
1      (12147)     (10348)            0.105263            0.098246  0.056140   
2      (10348)     (25922)            0.098246            0.098246  0.056140   
3      (25922)     (10348)            0.098246            0.098246  0.056140   
4      (40310)     (10348)            0.122807            0.098246  0.052632   
5      (10348)     (40310)            0.098246            0.122807  0.052632   
6      (10454)     (40310)            0.0771

D:\dev\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


根据结果可以看出，频繁项集frequent_itemsets中support是指某个集合在所有事务中出现的频率，而itemsets是指支持度大于等于最小支持度的集合。
在关联规则中，antecedents和consequents分别指规则的前件和后件，即规则中的条件和结论。antecedent support和consequent support分别指前件和后件在所有事务中出现的频率，而support是指同时出现的频率，confidence、lift、leverage、conviction和zhangs_metric参数分别是指置信度、提升度、杠杆率、确信度和Zhang’s metric。

4.  结果评估: 计算关联规则的支持度、置信度和提升度，得出强关联规则。

支持度、置信度、提升度的计算结果如下：

In [26]:
print("支持度：")
print(rules["support"])

print("置信度：")
print(rules["confidence"])

print("提升度：")
print(rules["lift"])

支持度：
0     0.056140
1     0.056140
2     0.056140
3     0.056140
4     0.052632
5     0.052632
6     0.052632
7     0.052632
8     0.059649
9     0.059649
10    0.052632
11    0.052632
12    0.052632
13    0.052632
14    0.052632
15    0.052632
16    0.056140
17    0.056140
18    0.052632
19    0.052632
20    0.056140
21    0.056140
22    0.052632
23    0.052632
Name: support, dtype: float64
置信度：
0     0.571429
1     0.533333
2     0.571429
3     0.571429
4     0.428571
5     0.535714
6     0.681818
7     0.428571
8     0.485714
9     0.566667
10    0.681818
11    0.535714
12    0.483871
13    0.428571
14    0.681818
15    0.535714
16    0.457143
17    0.571429
18    0.535714
19    0.535714
20    0.695652
21    0.457143
22    0.535714
23    0.428571
Name: confidence, dtype: float64
提升度：
0     5.428571
1     5.428571
2     5.816327
3     5.816327
4     4.362245
5     4.362245
6     5.551948
7     5.551948
8     4.614286
9     4.614286
10    6.939935
11    6.939935
12    3.940092
13    3

在Python中，使用Apriori算法来计算关联规则的支持度、置信度和提升度，并总结出其中的强关联规则。如果关联规则X->Y的支持度和置信度分别大于或等于用户指定的最小支持率minsupport和最小置信度minconfidence，则称关联规则X->Y为强关联规则，否则称关联规则X->Y为弱关联规则。
根据计算结果可以看出，其中规则满足支持度和置信度大于或等于minsupport=0.05和最小置信度minconfidence=0.5的规则都是强关联规则。
强关联规则如下：

In [28]:
#输出强关联规则
h_rules = rules.loc[(rules['support'] > 0.05) & (rules['confidence'] > 0.5)]
print('强关联规则：\n',h_rules.sort_values(by='support'))

强关联规则：
    antecedents consequents  antecedent support  consequent support   support  \
5      (10348)     (40310)            0.098246            0.122807  0.052632   
6      (10454)     (40310)            0.077193            0.122807  0.052632   
10     (23157)     (25922)            0.077193            0.098246  0.052632   
11     (25922)     (23157)            0.098246            0.077193  0.052632   
14     (33341)     (25922)            0.077193            0.098246  0.052632   
15     (25922)     (33341)            0.098246            0.077193  0.052632   
18     (41066)     (25922)            0.098246            0.098246  0.052632   
19     (25922)     (41066)            0.098246            0.098246  0.052632   
22     (41066)     (40310)            0.098246            0.122807  0.052632   
0      (10348)     (12147)            0.098246            0.105263  0.056140   
1      (12147)     (10348)            0.105263            0.098246  0.056140   
2      (10348)     (25922)      

5.  结果分析与应用: 分析得到的关联规则，为网站提供导航结构优化建议，以提升用户体验。

针对上述得出的强关联规则，以第一行为例，可以看出，（10348）->（40310）为强关联规则，说明用户访问这两个网站的关联性较强，可以按照关联规则向用户推荐常访问的网站信息，并提供网站跳转链接信息。

In [ ]:
    antecedents consequents  antecedent support  consequent support   support  \
5      (10348)     (40310)            0.098246            0.122807  0.052632  
    print()